# CRUD simples no Python utilizando SQL Server.

Para poder testar esse código, altere o ```config.ini``` e crie um banco chamado BandoDeClientes.

In [12]:
import pyodbc
import configparser

In [13]:
# Lendo o arquivo ini.
config = configparser.ConfigParser()
config.read('config.ini')

# Colocando as chaves correspondem do arquivo. 
chaves = ['Driver', 'Server', 'Database', 'Trusted_Connection']

valores_conexao = {}

for chave in chaves:
    valores_conexao[chave] = config.get('conexaoSQL', chave)

dados_conexao = (
    f"Driver={valores_conexao['Driver']};"
    f"Server={valores_conexao['Server']};"
    f"Database={valores_conexao['Database']};"
    f"Trusted_Connection={valores_conexao['Trusted_Connection']};"
)

{'Driver': '{SQL Server}', 'Server': 'DESKTOP-UMMM2O6\\SQLEXPRESS', 'Database': 'BancoDeClientes', 'Trusted_Connection': 'yes'}


In [14]:
tabela_informacao_cliente = """
    CREATE TABLE informacao_cliente (
        CPF VARCHAR(11) PRIMARY KEY, 
        Nome VARCHAR(255),
        Endereco VARCHAR(255),
        Telefone VARCHAR(255)
    )
"""

In [15]:
# Tentando estabelecer a conexão
try:
    conexao = pyodbc.connect(dados_conexao)
    cursor = conexao.cursor()

   # Verificando se a tabela informacao_cliente existe.
    cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'informacao_cliente'")
    existe_informacao_cliente = cursor.fetchone() is not None

    # Se existir, ele irá deletá-las. 
    if existe_informacao_cliente:
        cursor.execute("DROP TABLE informacao_cliente")
        print("Tabela existente excluída.")
        
    
    cursor.execute(tabela_informacao_cliente)
    conexao.commit()
    print("Tabela criada com sucesso!")
    conexao.close()

except pyodbc as e:
    print(f"Erro ao conectar ao banco de dados: {str(e)}")


Tabela existente excluída.
Tabela criada com sucesso!


### CREATE

In [16]:
def criar_cliente(cpf, nome, endereco, telefone):
    try:
        conexao = pyodbc.connect(dados_conexao)
        cursor = conexao.cursor()

        # Inserir um novo cliente
        cursor.execute("""INSERT INTO informacao_cliente 
                       (CPF, Nome, Endereco, Telefone) 
                       VALUES 
                       (?, ?, ?, ?)
                       """, cpf, nome, endereco, telefone)
        conexao.commit()
        print("Cliente criado com sucesso!")

    except pyodbc.Error as e:
        print(f"Erro ao criar cliente: {str(e)}")

### READ

In [17]:
def ler_cliente(cpf):
    try:
        conexao = pyodbc.connect(dados_conexao)
        cursor = conexao.cursor()

        # Através do CPF traz todas as informações do cliente.
        cursor.execute("""SELECT * FROM informacao_cliente 
                       WHERE 
                       CPF = ?
                       """,
                         cpf)
        
    
        cpf_existe = cursor.fetchone()

        #Se o CPF existe, então irá trazer de volta, caso contrário, irá retornar que não existe.
        if cpf_existe:
            print(f"""
                    CPF: {cpf_existe.CPF}
                    Nome: {cpf_existe.Nome}, 
                    Endereço: {cpf_existe.Endereco},
                    Telefone: {cpf_existe.Telefone}""")
        else:
            print("Não há nenhum cliente com esse CPF.")

    except pyodbc.Error as e:
        print(f"Erro ao ler cliente: {str(e)}")

### UPDATE

In [18]:
def atualizar_cliente(cpf, nome=None, endereco=None, telefone=None):
    try:
        conexao = pyodbc.connect(dados_conexao)
        cursor = conexao.cursor()

        # Query para atualizar o cadastro.
        consulta_sql = "UPDATE informacao_cliente SET "
        valores = []

        # Criando uma tupla para cada valor fornecido. 
        atualizar_campos = [("Nome", nome), ("Endereco", endereco), ("Telefone", telefone)]

        # Ver a quantidade de campos selecionados e atualizar os campos fornecidos.
        for campo, valor in atualizar_campos:
            if valor is not None:
                consulta_sql += f"{campo} = ?, "
                valores.append(valor)
        
        # Removendo a ultima virgula que é adicionada toda vez que o loop entra. 
        consulta_sql = consulta_sql.rstrip(", ")


        # Adicionando o WHERE no Final com o CPF fornecido inicialmente.
        consulta_sql += " WHERE CPF = ?"
        valores.append(cpf)
        print(consulta_sql)
        print(valores)


        cursor.execute(consulta_sql, *valores)
        conexao.commit()
        print("Cliente atualizado com sucesso!")

    except pyodbc.Error as e:
        print(f"Erro ao atualizar cliente: {str(e)}")

### DELETE

In [20]:
def deletar_cliente(cpf):
    try:
        conexao = pyodbc.connect(dados_conexao)
        cursor = conexao.cursor()

        # Deletar um cliente com base no CPF
        cursor.execute("DELETE FROM informacao_cliente WHERE CPF = ?", cpf)
        conexao.commit()
        print("Cliente deletado com sucesso!")

    except pyodbc.Error as e:
        print(f"Erro ao deletar cliente: {str(e)}")